<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/%ED%81%B4%EB%A6%B0%EB%A3%B8_HVAC_%EC%B5%9C%EC%A0%81%ED%99%94_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_(%EB%8D%B8%ED%83%80_%EC%97%94%ED%83%88%ED%94%BC_%EC%A0%81%EC%9A%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

# --- 0. 유틸리티 함수 (엔탈피 계산) ---
def calculate_humidity_ratio(temp_c, rel_hum_percent, p_atm_pa=101325):
    if pd.isna(temp_c) or pd.isna(rel_hum_percent) or rel_hum_percent < 0 or rel_hum_percent > 100:
        return np.nan
    p_ws_pa = 611.21 * np.exp((18.678 - temp_c / 234.5) * (temp_c / (257.14 + temp_c)))
    p_w_pa = (rel_hum_percent / 100) * p_ws_pa
    if p_atm_pa <= p_w_pa:
        return np.nan
    humidity_ratio = 0.621945 * p_w_pa / (p_atm_pa - p_w_pa)
    return humidity_ratio

def calculate_enthalpy(temp_c, humidity_ratio):
    if pd.isna(temp_c) or pd.isna(humidity_ratio): return np.nan
    enthalpy = 1.006 * temp_c + humidity_ratio * (2501 + 1.86 * temp_c)
    return enthalpy

class CleanroomHVACOptimizer:
    def __init__(self, num_oacs=14, num_low_chillers=8, num_high_chillers=8, num_cooling_towers=9):
        self.num_oacs = num_oacs
        self.num_low_chillers = num_low_chillers
        self.num_high_chillers = num_high_chillers
        self.num_cooling_towers = num_cooling_towers

        self.oac_scalers_X = {}
        self.oac_models = {}
        self.oac_feature_names = {}
        self.oac_target_names = {}

        self.low_ch_scaler_X = None
        self.low_ch_model = None
        self.low_ch_feature_names = []
        self.low_ch_target_names = []

        self.high_ch_scaler_X = None
        self.high_ch_model = None
        self.high_ch_feature_names = []
        self.high_ch_target_names = []

        self.rated_capacity_per_low_chiller = 700
        self.rated_capacity_per_high_chiller = 800
        self.rated_heat_rejection_per_ct = 1200

        self.optimal_load_ratio_min = 0.40
        self.optimal_load_ratio_max = 0.85

    def _prepare_oac_data_single(self, df_full_input, oac_id_num):
        """
        지정된 OAC ID에 대한 학습용 X, y 데이터를 준비합니다.
        각 코일의 입구 엔탈피와 델타 엔탈피(후단-전단)를 피처로 추가합니다.
        """
        df_full = df_full_input.copy() # 원본 변경 방지

        # 0. 기본 외기 엔탈피 계산
        if '외기_엔탈피' not in df_full.columns:
            df_full['외기_습도비'] = df_full.apply(lambda row: calculate_humidity_ratio(row['외기_온도'], row['외기_습도']), axis=1)
            df_full['외기_엔탈피'] = df_full.apply(lambda row: calculate_enthalpy(row['외기_온도'], row['외기_습도비']), axis=1)

        oac_col_prefix = f"OAC{oac_id_num}_"

        # 각 코일별 전/후단 엔탈피 및 델타 엔탈피 계산
        # 코일 처리 순서: 예열 -> 예냉 -> 냉각 -> 승온
        # 가습 코일은 후단 온/습도 데이터 부재로 델타 엔탈피 계산에서 제외 (필요시 추가 데이터 가정 필요)

        coil_stages = {
            '예열': {'in_temp_col': '외기_온도', 'in_hum_col': '외기_습도', 'in_enth_col': '외기_엔탈피'},
            '예냉': {'prev_coil': '예열'},
            '냉각': {'prev_coil': '예냉'},
            '승온': {'prev_coil': '냉각'}
        }

        calculated_enthalpy_features = {} # 계산된 엔탈피 관련 피처 임시 저장

        for coil_name, config in coil_stages.items():
            current_coil_prefix = f"{oac_col_prefix}{coil_name}_"

            # 1. 입구(전단) 엔탈피 결정
            in_enthalpy_col_name = f"{current_coil_prefix}전단엔탈피"
            if 'prev_coil' in config: # 이전 코일의 후단이 현재 코일의 전단
                prev_coil_stage_name = config['prev_coil']
                # 이전 코일의 후단 엔탈피 컬럼명이 calculated_enthalpy_features에 저장되어 있어야 함
                prev_coil_out_enthalpy_col = calculated_enthalpy_features.get(f"{oac_col_prefix}{prev_coil_stage_name}_후단엔탈피_컬럼명")
                if prev_coil_out_enthalpy_col and prev_coil_out_enthalpy_col in df_full.columns:
                    df_full[in_enthalpy_col_name] = df_full[prev_coil_out_enthalpy_col]
                else: # 이전 코일 후단 엔탈피 계산 불가 시, 현재 코일 처리 중단 (또는 NaN으로 채움)
                    df_full[in_enthalpy_col_name] = np.nan
                    # print(f"경고: OAC {oac_id_num} - {coil_name} 코일의 전단 엔탈피 계산 불가 (이전 코일 {prev_coil_stage_name} 후단 엔탈피 누락).")
            elif 'in_enth_col' in config: # 첫 코일 (예열)
                 if config['in_enth_col'] in df_full.columns:
                    df_full[in_enthalpy_col_name] = df_full[config['in_enth_col']]
                 else: # 외기 엔탈피 누락 시
                    df_full[in_enthalpy_col_name] = np.nan
                    # print(f"경고: OAC {oac_id_num} - {coil_name} 코일의 전단 엔탈피 계산 불가 ({config['in_enth_col']} 누락).")
            else: # 설정 오류
                df_full[in_enthalpy_col_name] = np.nan


            # 2. 출구(후단) 엔탈피 계산 (후단 온도/습도 데이터 필요)
            out_temp_col = f"{current_coil_prefix}후단_온도"
            out_hum_col = f"{current_coil_prefix}후단_습도"
            out_enthalpy_col_name = f"{current_coil_prefix}후단엔탈피"

            if out_temp_col in df_full.columns and out_hum_col in df_full.columns:
                # 후단 습도비 계산
                temp_out_hr_col = f"{current_coil_prefix}후단_습도비_임시" # 임시 컬럼
                df_full[temp_out_hr_col] = df_full.apply(lambda row: calculate_humidity_ratio(row[out_temp_col], row[out_hum_col]), axis=1)
                df_full[out_enthalpy_col_name] = df_full.apply(lambda row: calculate_enthalpy(row[out_temp_col], row[temp_out_hr_col]), axis=1)
                df_full.drop(columns=[temp_out_hr_col], inplace=True) # 임시 컬럼 삭제
                calculated_enthalpy_features[f"{current_coil_prefix}후단엔탈피_컬럼명"] = out_enthalpy_col_name # 다음 코일에서 사용하기 위해 저장
            else:
                df_full[out_enthalpy_col_name] = np.nan
                # print(f"경고: OAC {oac_id_num} - {coil_name} 코일의 후단 엔탈피 계산 불가 (후단 온도/습도 컬럼 누락: {out_temp_col} 또는 {out_hum_col}).")

            # 3. 델타 엔탈피 계산
            delta_h_col_name = f"{current_coil_prefix}델타엔탈피"
            if in_enthalpy_col_name in df_full.columns and out_enthalpy_col_name in df_full.columns:
                df_full[delta_h_col_name] = df_full[out_enthalpy_col_name] - df_full[in_enthalpy_col_name]
            else:
                df_full[delta_h_col_name] = np.nan

        # 입력 피처 구성: 기존 피처 + 각 코일의 전단 엔탈피 + 각 코일의 델타 엔탈피
        features = [
            '외기_온도', '외기_습도', '외기_엔탈피', # 기본 외기 조건
            f'{oac_col_prefix}FAN_Hz' # FAN 속도
        ]
        # 각 코일별 set_point, 전단엔탈피, 델타엔탈피 추가
        for coil_name in coil_stages.keys(): # 예열, 예냉, 냉각, 승온
            current_coil_prefix = f"{oac_col_prefix}{coil_name}_"
            features.append(f"{current_coil_prefix}set_point")
            features.append(f"{current_coil_prefix}전단엔탈피") # 학습 시 사용
            features.append(f"{current_coil_prefix}델타엔탈피") # 학습 시 사용

        # 가습 코일은 set_point만 (델타 엔탈피 계산 안함)
        features.append(f'{oac_col_prefix}가습_set_point')

        # 타겟 피처 (코일 개도율)
        targets = [
            f'{oac_col_prefix}예열_개도율', f'{oac_col_prefix}예냉_개도율',
            f'{oac_col_prefix}냉각_개도율', f'{oac_col_prefix}승온_개도율',
            f'{oac_col_prefix}가습_개도율'
        ]

        # 실제 데이터프레임에 존재하는 컬럼만 선택 (중요)
        valid_features = [f for f in features if f in df_full.columns]
        valid_targets = [t for t in targets if t in df_full.columns]

        # 필수 피처 누락 시 처리 (예: 전단엔탈피, 델타엔탈피 컬럼이 생성되지 않은 경우)
        # 생성된 엔탈피 관련 피처들이 valid_features에 포함되어 있는지 확인
        expected_enthalpy_features_count = len(coil_stages) * 2 # 각 코일당 전단H, 델타H
        actual_enthalpy_features_in_valid = sum(1 for f in valid_features if "전단엔탈피" in f or "델타엔탈피" in f)

        if not valid_features or not valid_targets or len(valid_targets) != len(targets) or actual_enthalpy_features_in_valid < expected_enthalpy_features_count:
            # print(f"경고: OAC {oac_id_num} 데이터 준비 중 컬럼 부족 또는 엔탈피 피처 생성 실패.")
            # print(f"  요구 피처 수 (근사): {len(features)}, 실제 유효 피처 수: {len(valid_features)}")
            # print(f"  요구 타겟 수: {len(targets)}, 실제 유효 타겟 수: {len(valid_targets)}")
            # print(f"  예상 엔탈피 피처 수: {expected_enthalpy_features_count}, 실제 생성된 유효 엔탈피 피처 수: {actual_enthalpy_features_in_valid}")
            # print(f"  유효 피처 목록: {valid_features}")
            return None, None, [], []

        X = df_full[valid_features].copy()
        y = df_full[valid_targets].copy()

        # 결측치 처리 (중앙값 사용)
        X.fillna(X.median(), inplace=True)
        y.fillna(y.median(), inplace=True)

        return X, y, valid_features, valid_targets

    def train_oac_models(self, df_historical_data):
        print("OAC 모델 학습 시작 (델타 엔탈피 적용)...")
        temp_df = df_historical_data.copy()
        # _prepare_oac_data_single 내부에서 외기 엔탈피 계산하므로 여기서 미리 할 필요 없음

        for i in range(1, self.num_oacs + 1):
            X_oac, y_oac, features, targets = self._prepare_oac_data_single(temp_df, i) # 함수 내부에서 엔탈피 피처 생성
            if X_oac is None or X_oac.empty:
                print(f"  OAC {i} 모델 학습 건너<0xEB><0><0x84>니다 (데이터 준비 실패 또는 피처 부족).")
                continue

            self.oac_feature_names[i] = features
            self.oac_target_names[i] = targets
            scaler = StandardScaler()
            X_oac_scaled = scaler.fit_transform(X_oac)
            self.oac_scalers_X[i] = scaler

            X_train, _, y_train, _ = train_test_split(X_oac_scaled, y_oac, test_size=0.2, shuffle=False, random_state=42)
            model = MultiOutputRegressor(RandomForestRegressor(n_estimators=70, random_state=42, n_jobs=-1, max_depth=20, min_samples_leaf=3, min_samples_split=6)) # 파라미터 약간 상향
            model.fit(X_train, y_train)
            self.oac_models[i] = model
        print("OAC 모델 학습 완료.")

    def predict_oac_coil_opening_ratios(self, current_outdoor_conditions_dict, oac_setpoints_list_of_dicts, modified_coil_info=None):
        """
        OAC 코일 개도율 예측. 학습 시 델타H를 사용했지만, 예측 시에는 델타H를 직접 입력받지 않음.
        대신, 델타H를 결정하는 요인인 '전단 엔탈피'와 'set_point'를 입력으로 사용.
        모델은 이 관계를 학습했다고 가정.
        """
        all_oac_predictions = {}
        outdoor_temp = current_outdoor_conditions_dict['외기_온도']
        outdoor_humidity = current_outdoor_conditions_dict['외기_습도']
        outdoor_hr = calculate_humidity_ratio(outdoor_temp, outdoor_humidity)
        outdoor_enthalpy = calculate_enthalpy(outdoor_temp, outdoor_hr)

        current_base_conditions = { # 모든 OAC에 공통적인 기본 조건
            '외기_온도': outdoor_temp,
            '외기_습도': outdoor_humidity,
            '외기_엔탈피': outdoor_enthalpy
        }

        for i in range(1, self.num_oacs + 1): # 각 OAC에 대해 순회
            if i not in self.oac_models or i not in self.oac_scalers_X or not self.oac_feature_names.get(i):
                target_names_for_oac = self.oac_target_names.get(i, [f"OAC{i}_코일{j}_개도율" for j in range(1,6)])
                all_oac_predictions[i] = {name: np.nan for name in target_names_for_oac}
                continue

            # 현재 OAC의 설정값 가져오기
            current_oac_setpoints = {}
            if i-1 < len(oac_setpoints_list_of_dicts):
                 current_oac_setpoints = oac_setpoints_list_of_dicts[i-1] # 예: {'OAC1_FAN_Hz': 50, 'OAC1_예열_set_point': 20 ...}

            # 예측을 위한 입력 피처 구성 (학습 시 사용된 피처 순서대로)
            # '델타엔탈피' 피처는 예측 시점에는 알 수 없으므로, 모델이 다른 피처들로부터 이를 추론하도록 학습됨.
            # 따라서 예측 입력에서는 '델타엔탈피' 컬럼은 제외하고, '전단엔탈피'는 계산해서 넣어줌.

            input_feature_values_dict = current_base_conditions.copy()
            input_feature_values_dict[f"OAC{i}_FAN_Hz"] = current_oac_setpoints.get(f"OAC{i}_FAN_Hz", np.nan) # FAN_Hz는 setpoint에서 가져옴

            # 코일별 전단 엔탈피 순차 계산 및 set_point 추가
            # 이 부분은 예측 시에도 각 코일의 '전단엔탈피'를 계산해서 모델 입력으로 넣어줘야 함.
            # 하지만, '후단 상태'를 알아야 다음 코일의 '전단 엔탈피'를 알 수 있는데,
            # '후단 상태'는 현재 예측하려는 '개도율'의 결과임. -> 순환적 의존성 발생.
            #
            # 해결책: MultiOutputRegressor가 모든 코일 개도율을 동시에 예측하므로,
            # 각 코일의 '전단엔탈피'를 예측 시점에서 정확히 알기 어려움.
            #
            # 가장 간단한 접근: 학습 시에는 실제 전단H, 델타H를 사용했지만,
            # 예측 시에는 '전단엔탈피'와 '델타엔탈피' 피처를 제외하고,
            # 모델이 외기조건, FAN_Hz, 각 코일 set_point 만으로 개도율을 예측하도록 하는 것.
            # 이 경우, _prepare_oac_data_single에서 학습 피처와 예측 시 피처가 달라지는 문제 발생.
            #
            # 차선책: 예측 시 '전단엔탈피'는 이전 단계의 set_point 기반으로 "목표 후단 엔탈피"를 추정하여 사용.
            #          '델타엔탈피'는 (목표 후단 엔탈피 - 전단 엔탈피)로 추정.
            #          이것도 정확도가 떨어질 수 있음.
            #
            # 이번 구현에서는, 학습 시 사용한 피처 중 '델타엔탈피'는 예측 시점에 알 수 없으므로,
            # 해당 피처를 제외한 나머지 피처들로 예측을 시도하도록 모델을 구성하거나,
            # 또는 '델타엔탈피'를 0 또는 평균값 등으로 채워서 입력해야 함.
            #
            # 여기서는 학습 피처와 예측 피처를 일치시키기 위해,
            # 예측 시 '전단엔탈피'는 이전 코일의 '목표 후단 엔탈피'(set_point 기반)로,
            # '델타엔탈피'는 '목표 델타 엔탈피'(set_point - 전단온도에 해당하는 엔탈피 차이)로 근사하여 입력.
            # 이는 정확한 방법은 아니지만, 피처 수를 맞추기 위한 임시 방편.
            #
            # 더 나은 방법은 모델 구조 변경 또는 피처 엔지니어링 재검토.
            # 지금은 _prepare_oac_data_single에서 학습 피처로 전단H와 델타H를 넣었으므로, 예측 시에도 이 값들을 "추정"해서 넣어야 함.

            # --- 예측 시 전단 엔탈피 및 (목표) 델타 엔탈피 추정 로직 ---
            # 이 부분은 매우 단순화된 추정이며, 실제 정확도를 위해서는 개선 필요.
            oac_prefix_for_sp = f"OAC{i}_" # setpoint 딕셔너리 키용 접두사

            # 1. 예열 코일
            h_in_preheat = outdoor_enthalpy
            input_feature_values_dict[f"OAC{i}_예열_전단엔탈피"] = h_in_preheat
            sp_preheat_temp = current_oac_setpoints.get(f"{oac_prefix_for_sp}예열_set_point", outdoor_temp) # SP 없으면 외기온도
            # 목표 후단 엔탈피 (단순히 온도만으로 추정, 습도는 외기 습도 유지 가정 - 매우 부정확)
            h_out_target_preheat = calculate_enthalpy(sp_preheat_temp, outdoor_hr if pd.notna(outdoor_hr) else 0.01)
            input_feature_values_dict[f"OAC{i}_예열_델타엔탈피"] = (h_out_target_preheat - h_in_preheat) if pd.notna(h_out_target_preheat) and pd.notna(h_in_preheat) else 0
            input_feature_values_dict[f"OAC{i}_예열_set_point"] = sp_preheat_temp

            # 2. 예냉 코일 (예열 코일의 목표 후단이 예냉 코일의 전단으로 가정)
            h_in_precool = h_out_target_preheat if pd.notna(h_out_target_preheat) else outdoor_enthalpy # 이전 단계 목표 후단
            input_feature_values_dict[f"OAC{i}_예냉_전단엔탈피"] = h_in_precool
            sp_precool_temp = current_oac_setpoints.get(f"{oac_prefix_for_sp}예냉_set_point", sp_preheat_temp)
            h_out_target_precool = calculate_enthalpy(sp_precool_temp, outdoor_hr) # 습도 변화 없다고 가정
            input_feature_values_dict[f"OAC{i}_예냉_델타엔탈피"] = (h_out_target_precool - h_in_precool) if pd.notna(h_out_target_precool) and pd.notna(h_in_precool) else 0
            input_feature_values_dict[f"OAC{i}_예냉_set_point"] = sp_precool_temp

            # 3. 냉각 코일
            h_in_cool = h_out_target_precool if pd.notna(h_out_target_precool) else outdoor_enthalpy
            input_feature_values_dict[f"OAC{i}_냉각_전단엔탈피"] = h_in_cool
            sp_cool_temp = current_oac_setpoints.get(f"{oac_prefix_for_sp}냉각_set_point", sp_precool_temp)
            h_out_target_cool = calculate_enthalpy(sp_cool_temp, outdoor_hr) # 습도 변화 없다고 가정 (제습 고려 안함)
            input_feature_values_dict[f"OAC{i}_냉각_델타엔탈피"] = (h_out_target_cool - h_in_cool) if pd.notna(h_out_target_cool) and pd.notna(h_in_cool) else 0
            input_feature_values_dict[f"OAC{i}_냉각_set_point"] = sp_cool_temp

            # 4. 승온 코일
            h_in_reheat = h_out_target_cool if pd.notna(h_out_target_cool) else outdoor_enthalpy
            input_feature_values_dict[f"OAC{i}_승온_전단엔탈피"] = h_in_reheat
            sp_reheat_temp = current_oac_setpoints.get(f"{oac_prefix_for_sp}승온_set_point", sp_cool_temp)
            h_out_target_reheat = calculate_enthalpy(sp_reheat_temp, outdoor_hr)
            input_feature_values_dict[f"OAC{i}_승온_델타엔탈피"] = (h_out_target_reheat - h_in_reheat) if pd.notna(h_out_target_reheat) and pd.notna(h_in_reheat) else 0
            input_feature_values_dict[f"OAC{i}_승온_set_point"] = sp_reheat_temp

            # 가습 코일 set_point
            input_feature_values_dict[f"OAC{i}_가습_set_point"] = current_oac_setpoints.get(f"{oac_prefix_for_sp}가습_set_point", np.nan)


            # 최종 입력 데이터 리스트 생성 (학습 시 피처 순서와 동일하게)
            input_data_values = []
            for fn_template in self.oac_feature_names[i]: # 학습 시 사용된 피처 이름 순서대로 값 가져오기
                # fn_template 예: 'OAC1_예열_set_point', 'OAC1_예열_전단엔탈피', 'OAC1_예열_델타엔탈피' 등
                input_data_values.append(input_feature_values_dict.get(fn_template, np.nan))

            input_df = pd.DataFrame([input_data_values], columns=self.oac_feature_names[i])
            input_df.fillna(0, inplace=True) # NaN은 0으로 (또는 훈련 데이터의 중앙값/평균으로 채워야 함)

            input_scaled = self.oac_scalers_X[i].transform(input_df)
            prediction_array = self.oac_models[i].predict(input_scaled)
            oac_pred_dict = dict(zip(self.oac_target_names[i], prediction_array[0]))
            all_oac_predictions[i] = oac_pred_dict

        if modified_coil_info: # 수동 개도율 수정 로직 (동일)
            mod_oac_id = modified_coil_info['oac_id_num']
            mod_coil_name = modified_coil_info['coil_target_name']
            mod_value = modified_coil_info['value']
            if mod_oac_id in all_oac_predictions and mod_coil_name in all_oac_predictions[mod_oac_id]:
                original_value = all_oac_predictions[mod_oac_id][mod_coil_name]
                all_oac_predictions[mod_oac_id][mod_coil_name] = mod_value
                print(f"알림: OAC {mod_oac_id}의 {mod_coil_name} 개도율이 {original_value:.2f}%에서 {mod_value:.2f}%(으)로 수동 변경되었습니다.")
                print("      주의: 이 변경으로 인한 다른 코일들의 자동 연동 조정은 현재 단순 덮어쓰기 방식입니다.")
            else:
                print(f"경고: 수정하려는 코일 정보(OAC ID: {mod_oac_id}, 코일명: {mod_coil_name})를 찾을 수 없습니다.")
        return all_oac_predictions

    # --- 이하 냉동기 및 냉각탑 관련 메소드는 이전과 거의 동일 ---
    def _prepare_chiller_data_common(self, df_full, chiller_type):
        temp_df = df_full.copy()
        if '외기_엔탈피' not in temp_df.columns and '외기_온도' in temp_df.columns and '외기_습도' in temp_df.columns:
             temp_df['외기_습도비'] = temp_df.apply(lambda row: calculate_humidity_ratio(row['외기_온도'], row['외기_습도']), axis=1)
             temp_df['외기_엔탈피'] = temp_df.apply(lambda row: calculate_enthalpy(row['외기_온도'], row['외기_습도비']), axis=1)

        relevant_coil_suffix = '_예냉_개도율' if chiller_type == 'high' else '_냉각_개도율'
        total_relevant_coil_opening = pd.Series(0.0, index=temp_df.index)
        for oac_id in range(1, self.num_oacs + 1):
            oac_col_prefix = f"OAC{oac_id}_"
            coil_col_name = f"{oac_col_prefix}{relevant_coil_suffix.lstrip('_')}"
            if coil_col_name in temp_df.columns:
                fan_hz_col = f"{oac_col_prefix}FAN_Hz"
                weight = temp_df[fan_hz_col].fillna(temp_df[fan_hz_col].median()) if fan_hz_col in temp_df.columns else 1
                total_relevant_coil_opening += temp_df[coil_col_name].fillna(0) * weight
        temp_df[f'Total_{chiller_type}_Coil_Load_Indicator'] = total_relevant_coil_opening

        features = [f'Total_{chiller_type}_Coil_Load_Indicator', '외기_온도', '외기_습도', '외기_엔탈피']
        targets_map = {
            'low': ['저온냉동기_메인차압압력', '저온냉동기_메인차압개도율', '저온냉동기_서브차압개도율'],
            'high': ['고온냉동기_메인차압압력', '고온냉동기_메인차압개도율', '고온냉동기_서브차압개도율']
        }
        targets = targets_map[chiller_type]
        valid_features = [f for f in features if f in temp_df.columns]
        valid_targets = [t for t in targets if t in temp_df.columns]

        if not valid_features or len(valid_features) != len(features) or \
           not valid_targets or len(valid_targets) != len(targets):
            return None, None, [], []

        X = temp_df[valid_features].copy()
        y = temp_df[valid_targets].copy()
        X.fillna(X.median(), inplace=True)
        y.fillna(y.median(), inplace=True)
        return X, y, valid_features, valid_targets

    def train_chiller_models(self, df_historical_data):
        print("냉동기 모델 학습 시작...")
        temp_hist_data = df_historical_data.copy()
        if '외기_엔탈피' not in temp_hist_data.columns:
            temp_hist_data['외기_습도비'] = temp_hist_data.apply(lambda row: calculate_humidity_ratio(row['외기_온도'], row['외기_습도']), axis=1)
            temp_hist_data['외기_엔탈피'] = temp_hist_data.apply(lambda row: calculate_enthalpy(row['외기_온도'], row['외기_습도비']), axis=1)

        for chiller_type, model_attr_prefix in [('low', 'low_ch'), ('high', 'high_ch')]:
            X_ch, y_ch, f_ch, t_ch = self._prepare_chiller_data_common(temp_hist_data, chiller_type)
            if X_ch is None:
                print(f"  {chiller_type.capitalize()} 온도 냉동기 모델 학습 건너<0xEB><0><0x84>니다.")
                continue

            setattr(self, f"{model_attr_prefix}_feature_names", f_ch)
            setattr(self, f"{model_attr_prefix}_target_names", t_ch)
            scaler = StandardScaler().fit(X_ch)
            setattr(self, f"{model_attr_prefix}_scaler_X", scaler)
            X_ch_scaled = scaler.transform(X_ch)
            X_train, _, y_train, _ = train_test_split(X_ch_scaled, y_ch, test_size=0.2, shuffle=False, random_state=42)
            model = MultiOutputRegressor(RandomForestRegressor(n_estimators=60, random_state=42, n_jobs=-1, max_depth=12, min_samples_leaf=5))
            model.fit(X_train, y_train)
            setattr(self, f"{model_attr_prefix}_model", model)
        print("냉동기 모델 학습 완료.")

    def predict_chiller_parameters(self, all_oac_coil_predictions_dict, current_outdoor_conditions_dict):
        chiller_predictions_output = {'low_temp_chiller': {}, 'high_temp_chiller': {}}
        outdoor_temp = current_outdoor_conditions_dict['외기_온도']
        outdoor_humidity = current_outdoor_conditions_dict['외기_습도']
        outdoor_hr = calculate_humidity_ratio(outdoor_temp, outdoor_humidity)
        outdoor_enthalpy = calculate_enthalpy(outdoor_temp, outdoor_hr)
        current_outdoor_plus_enthalpy = {**current_outdoor_conditions_dict, '외기_엔탈피': outdoor_enthalpy}

        for chiller_type, model_attr_prefix in [('low', 'low_ch'), ('high', 'high_ch')]:
            model = getattr(self, f"{model_attr_prefix}_model")
            scaler_X = getattr(self, f"{model_attr_prefix}_scaler_X")
            feature_names = getattr(self, f"{model_attr_prefix}_feature_names")
            target_names = getattr(self, f"{model_attr_prefix}_target_names")

            if not model or not scaler_X or not feature_names:
                chiller_predictions_output[f"{chiller_type}_temp_chiller"] = {tn: np.nan for tn in target_names if isinstance(tn, str)} # target_names가 리스트여야 함
                chiller_predictions_output[f"{chiller_type}_temp_chiller"]['calculated_total_load_indicator'] = np.nan
                continue

            total_load_indicator = 0
            coil_suffix_keyword = '냉각_개도율' if chiller_type == 'low' else '예냉_개도율'

            for oac_id in range(1, self.num_oacs + 1):
                oac_preds = all_oac_coil_predictions_dict.get(oac_id, {})
                relevant_coil_key = next((k for k in oac_preds.keys() if coil_suffix_keyword in k), None)
                if relevant_coil_key:
                    total_load_indicator += oac_preds.get(relevant_coil_key, 0)

            input_data = {f'Total_{chiller_type}_Coil_Load_Indicator': total_load_indicator, **current_outdoor_plus_enthalpy}
            input_df_list = [input_data.get(fn, np.nan) for fn in feature_names]

            input_df = pd.DataFrame([input_df_list], columns=feature_names)
            input_df.fillna(0, inplace=True)

            input_df_scaled = scaler_X.transform(input_df)
            pred_array = model.predict(input_df_scaled)

            chiller_predictions_output[f'{chiller_type}_temp_chiller'] = dict(zip(target_names, pred_array[0]))
            chiller_predictions_output[f'{chiller_type}_temp_chiller']['calculated_total_load_indicator'] = total_load_indicator
        return chiller_predictions_output

    def _calculate_optimized_units(self, total_load, capacity_per_unit, max_units, min_load_ratio, max_load_ratio):
        if pd.isna(total_load) or total_load <= 0 or capacity_per_unit <= 0:
            return 0
        min_required_units = int(np.ceil(total_load / capacity_per_unit))
        min_required_units = min(max(0, min_required_units), max_units)
        if min_required_units == 0: return 0
        best_units = min_required_units
        best_score = float('inf')
        for num_op_units in range(min_required_units, max_units + 1):
            if num_op_units == 0: continue
            current_total_capacity = num_op_units * capacity_per_unit
            if current_total_capacity < total_load and num_op_units < max_units : continue
            avg_load_per_unit_ratio = total_load / current_total_capacity
            if avg_load_per_unit_ratio > 1.0: continue
            if avg_load_per_unit_ratio < 0.05 and num_op_units > min_required_units : continue
            target_optimal_ratio = (min_load_ratio + max_load_ratio) / 2
            current_score = abs(avg_load_per_unit_ratio - target_optimal_ratio)
            if avg_load_per_unit_ratio < min_load_ratio and num_op_units > min_required_units:
                current_score += (min_load_ratio - avg_load_per_unit_ratio)
            if num_op_units == min_required_units:
                best_units = num_op_units
                best_score = current_score
            elif current_score < best_score :
                best_units = num_op_units
                best_score = current_score
            elif current_score == best_score and num_op_units < best_units:
                best_units = num_op_units
        return best_units

    def optimize_chiller_operating_units(self, chiller_parameter_predictions):
        low_ch_load = chiller_parameter_predictions.get('low_temp_chiller', {}).get('calculated_total_load_indicator', 0)
        num_optimal_low = self._calculate_optimized_units(
            low_ch_load, self.rated_capacity_per_low_chiller, self.num_low_chillers,
            self.optimal_load_ratio_min, self.optimal_load_ratio_max
        )
        high_ch_load = chiller_parameter_predictions.get('high_temp_chiller', {}).get('calculated_total_load_indicator', 0)
        num_optimal_high = self._calculate_optimized_units(
            high_ch_load, self.rated_capacity_per_high_chiller, self.num_high_chillers,
            self.optimal_load_ratio_min, self.optimal_load_ratio_max
        )
        return {
            "optimal_low_temp_chillers_count": num_optimal_low,
            "optimal_high_temp_chillers_count": num_optimal_high
        }

    def optimize_cooling_tower_operating_units(self, optimal_chiller_counts_dict, chiller_parameter_predictions):
        low_ch_actual_load = chiller_parameter_predictions.get('low_temp_chiller', {}).get('calculated_total_load_indicator', 0)
        high_ch_actual_load = chiller_parameter_predictions.get('high_temp_chiller', {}).get('calculated_total_load_indicator', 0)
        assumed_cop = 4.0
        total_heat_rejection_load = 0
        if optimal_chiller_counts_dict["optimal_low_temp_chillers_count"] > 0 and pd.notna(low_ch_actual_load) and low_ch_actual_load > 0:
            total_heat_rejection_load += low_ch_actual_load * (1 + 1/assumed_cop)
        if optimal_chiller_counts_dict["optimal_high_temp_chillers_count"] > 0 and pd.notna(high_ch_actual_load) and high_ch_actual_load > 0:
            total_heat_rejection_load += high_ch_actual_load * (1 + 1/assumed_cop)
        num_optimal_ct = self._calculate_optimized_units(
            total_heat_rejection_load, self.rated_heat_rejection_per_ct, self.num_cooling_towers,
            self.optimal_load_ratio_min, self.optimal_load_ratio_max
        )
        return {"optimal_cooling_towers_count": num_optimal_ct}

# --- 메인 실행 로직 ---
if __name__ == '__main__':
    print("향상된 더미 데이터 생성 및 HVAC 최적화 시뮬레이션 시작 (델타 엔탈피 적용)...")

    num_hours_in_year = 365 * 24
    datetime_index = pd.date_range(start='2023-01-01 00:00:00', periods=num_hours_in_year, freq='H')
    df_dummy_data = pd.DataFrame(index=datetime_index)

    df_dummy_data['hour_of_day'] = df_dummy_data.index.hour
    df_dummy_data['day_of_year'] = df_dummy_data.index.dayofyear
    df_dummy_data['month'] = df_dummy_data.index.month

    base_temp_annual = 12
    amplitude_annual = 15
    df_dummy_data['외기_온도'] = base_temp_annual + \
                             amplitude_annual * np.sin(2 * np.pi * (df_dummy_data['day_of_year'] - 90) / 365) + \
                             5 * np.sin(2 * np.pi * df_dummy_data['hour_of_day'] / 24) + \
                             np.random.normal(0, 1.5, num_hours_in_year)
    df_dummy_data['외기_온도'] = np.clip(df_dummy_data['외기_온도'], -15, 40)

    base_humidity_annual = 60
    amplitude_humidity_annual = 25
    month_effect_humidity = np.sin(2 * np.pi * (df_dummy_data['month'] - 4) / 12)
    df_dummy_data['외기_습도'] = base_humidity_annual + \
                              amplitude_humidity_annual * month_effect_humidity + \
                              10 * np.sin(2 * np.pi * df_dummy_data['hour_of_day'] / 24 + np.pi) + \
                              np.random.normal(0, 5, num_hours_in_year)
    df_dummy_data['외기_습도'] = np.clip(df_dummy_data['외기_습도'], 15, 99)

    # OAC 데이터 생성 시 후단 온/습도 포함 필수
    for i in range(1, 15):
        p = f"OAC{i}_"
        df_dummy_data[f'{p}FAN_Hz'] = np.random.uniform(35, 55, num_hours_in_year) + np.random.normal(0,2,num_hours_in_year)
        df_dummy_data[f'{p}FAN_Hz'] = np.clip(df_dummy_data[f'{p}FAN_Hz'], 30, 60)

        # 코일별 set_point 생성
        df_dummy_data[f'{p}예열_set_point'] = np.clip(20 - 0.3 * df_dummy_data['외기_온도'], 18, 25) + np.random.normal(0,0.5,num_hours_in_year)
        df_dummy_data[f'{p}예냉_set_point'] = np.clip(18 + 0.1 * df_dummy_data['외기_온도'], 16, 22) + np.random.normal(0,0.5,num_hours_in_year)
        df_dummy_data[f'{p}냉각_set_point'] = np.clip(15 + 0.05 * df_dummy_data['외기_온도'], 10, 18) + np.random.normal(0,0.5,num_hours_in_year)
        df_dummy_data[f'{p}승온_set_point'] = np.random.uniform(21, 24, num_hours_in_year) + np.random.normal(0,0.5,num_hours_in_year)
        df_dummy_data[f'{p}가습_set_point'] = np.random.uniform(40, 55, num_hours_in_year)

        # 코일별 후단 온도/습도 생성 (델타H 계산의 기반)
        # 예열 후단
        df_dummy_data[f'{p}예열_후단_온도'] = df_dummy_data[f'{p}예열_set_point'] + np.random.normal(0, 0.2, num_hours_in_year) # set_point 근처
        df_dummy_data[f'{p}예열_후단_습도'] = df_dummy_data['외기_습도'] + np.random.normal(0, 2, num_hours_in_year) # 외기 습도에서 크게 변하지 않음
        df_dummy_data[f'{p}예열_후단_습도'] = np.clip(df_dummy_data[f'{p}예열_후단_습도'],10,99)
        # 예냉 후단
        df_dummy_data[f'{p}예냉_후단_온도'] = df_dummy_data[f'{p}예냉_set_point'] + np.random.normal(0, 0.2, num_hours_in_year)
        df_dummy_data[f'{p}예냉_후단_습도'] = df_dummy_data[f'{p}예열_후단_습도'] - np.random.uniform(0,5,num_hours_in_year) # 약간의 제습 효과
        df_dummy_data[f'{p}예냉_후단_습도'] = np.clip(df_dummy_data[f'{p}예냉_후단_습도'],10,99)
        # 냉각 후단
        df_dummy_data[f'{p}냉각_후단_온도'] = df_dummy_data[f'{p}냉각_set_point'] + np.random.normal(0, 0.2, num_hours_in_year)
        df_dummy_data[f'{p}냉각_후단_습도'] = df_dummy_data[f'{p}예냉_후단_습도'] - np.random.uniform(1,8,num_hours_in_year) # 제습 효과 더 크게
        df_dummy_data[f'{p}냉각_후단_습도'] = np.clip(df_dummy_data[f'{p}냉각_후단_습도'],10,95) # 과도한 제습 방지
        # 승온 후단
        df_dummy_data[f'{p}승온_후단_온도'] = df_dummy_data[f'{p}승온_set_point'] + np.random.normal(0, 0.2, num_hours_in_year)
        df_dummy_data[f'{p}승온_후단_습도'] = df_dummy_data[f'{p}냉각_후단_습도'] # 승온 시 습도 변화 거의 없음

        # 코일별 개도율 생성 (이전과 유사하게, 단 델타H와의 관계는 모델이 학습)
        df_dummy_data[f'{p}예열_개도율'] = np.clip(50 - 2.5 * df_dummy_data['외기_온도'] + np.random.normal(0,10,num_hours_in_year), 0, 100)
        df_dummy_data[f'{p}냉각_개도율'] = np.clip(2.0 * (df_dummy_data['외기_온도'] - 15) + np.random.normal(0,10,num_hours_in_year), 0, 100)
        df_dummy_data[f'{p}예냉_개도율'] = np.clip(df_dummy_data[f'{p}냉각_개도율'] * np.random.uniform(0.3, 0.7), 0, 100)
        df_dummy_data[f'{p}승온_개도율'] = np.clip(np.abs(df_dummy_data[f'{p}냉각_개도율'] - df_dummy_data[f'{p}예열_개도율'])*0.3 + np.random.normal(0,5,num_hours_in_year) , 0, 100)
        df_dummy_data[f'{p}가습_개도율'] = np.clip(60 - df_dummy_data['외기_습도'] + np.random.normal(0,10,num_hours_in_year), 0, 100)

    # 냉동기 데이터 (이전과 동일)
    total_cooling_load_indicator = pd.Series(0.0, index=df_dummy_data.index)
    total_precooling_load_indicator = pd.Series(0.0, index=df_dummy_data.index)
    for i in range(1, 15):
        p = f"OAC{i}_"
        total_cooling_load_indicator += df_dummy_data[f'{p}냉각_개도율'].fillna(0) * df_dummy_data[f'{p}FAN_Hz'].fillna(40)
        total_precooling_load_indicator += df_dummy_data[f'{p}예냉_개도율'].fillna(0) * df_dummy_data[f'{p}FAN_Hz'].fillna(40)

    for prefix, load_indicator_series in [("저온냉동기", total_cooling_load_indicator), ("고온냉동기", total_precooling_load_indicator)]:
        main_p = prefix + "_"
        max_load = load_indicator_series.max()
        if max_load == 0: max_load = 1
        base_opening = (load_indicator_series / max_load) * 70 + 10
        df_dummy_data[f'{main_p}메인차압개도율'] = np.clip(base_opening + np.random.normal(0,5,num_hours_in_year), 5, 95)
        df_dummy_data[f'{main_p}서브차압개도율'] = np.clip(df_dummy_data[f'{main_p}메인차압개도율'] * np.random.uniform(0.7,1.1), 5, 95)
        df_dummy_data[f'{main_p}메인차압압력'] = df_dummy_data[f'{main_p}메인차압개도율'] * 0.015 + np.random.uniform(0.3,0.8)
        df_dummy_data[f'{main_p}메인supply압력'] = np.random.uniform(3, 7, num_hours_in_year)
        df_dummy_data[f'{main_p}메인supply온도'] = np.random.uniform(5, 9, num_hours_in_year) if "저온" in prefix else np.random.uniform(9, 13, num_hours_in_year)
        df_dummy_data[f'{main_p}메인return압력'] = df_dummy_data[f'{main_p}메인supply압력'] - np.random.uniform(0.5, 1.5, num_hours_in_year)
        df_dummy_data[f'{main_p}메인return온도'] = df_dummy_data[f'{main_p}메인supply온도'] + (df_dummy_data[f'{main_p}메인차압개도율']/100)*5 + np.random.uniform(1,3)
        df_dummy_data[f'{main_p}메인return온도'] = np.clip(df_dummy_data[f'{main_p}메인return온도'], df_dummy_data[f'{main_p}메인supply온도']+0.5, 30)

    for col in df_dummy_data.columns:
        if df_dummy_data[col].isnull().any():
            # print(f"컬럼 '{col}'에서 NaN 값을 중앙값으로 채웁니다.")
            df_dummy_data[col].fillna(df_dummy_data[col].median(), inplace=True)

    df_historical_data_improved = df_dummy_data.copy()
    print(f"향상된 더미 데이터 생성 완료: {df_historical_data_improved.shape[0]}개 행")
    # print("더미 데이터 NaN 체크:")
    # print(df_historical_data_improved.isnull().sum().sort_values(ascending=False).head(10))


    hvac_optimizer = CleanroomHVACOptimizer()

    print("\n=== 모델 학습 시작 (델타 엔탈피 적용) ===")
    hvac_optimizer.train_oac_models(df_historical_data_improved)
    hvac_optimizer.train_chiller_models(df_historical_data_improved)
    print("=== 모델 학습 완료 ===\n")

    print("=== 예측 및 최적화 실행 (예시 시나리오) ===")
    current_outdoor_conditions = {'외기_온도': 28.0, '외기_습도': 70.0}

    example_oac_setpoints_input = []
    for i in range(1, hvac_optimizer.num_oacs + 1):
        oac_sp_prefix = f"OAC{i}_"
        example_oac_setpoints_input.append({
            f'{oac_sp_prefix}FAN_Hz': 50.0, # FAN_Hz도 setpoint의 일부로 전달
            f'{oac_sp_prefix}예열_set_point': 18.0,
            f'{oac_sp_prefix}예냉_set_point': 16.0,
            f'{oac_sp_prefix}냉각_set_point': 13.0,
            f'{oac_sp_prefix}승온_set_point': 22.0,
            f'{oac_sp_prefix}가습_set_point': 50.0
        })

    print("\n--- OAC 코일 개도율 예측 ---")
    predicted_oac_coils = hvac_optimizer.predict_oac_coil_opening_ratios(
        current_outdoor_conditions, example_oac_setpoints_input
    )
    if 1 in predicted_oac_coils and predicted_oac_coils[1]:
        print("OAC 1 예측된 코일 개도율:")
        for coil_name, value in predicted_oac_coils[1].items():
            print(f"  {coil_name}: {value:.2f}%")

    print("\n--- 냉동기 파라미터 예측 ---")
    predicted_chiller_params = hvac_optimizer.predict_chiller_parameters(
        predicted_oac_coils, current_outdoor_conditions
    )
    print("예측된 냉동기 파라미터 및 총 부하 지표:")
    if predicted_chiller_params['low_temp_chiller']:
        print(f"  저온 냉동기: {predicted_chiller_params['low_temp_chiller']}")
    if predicted_chiller_params['high_temp_chiller']:
        print(f"  고온 냉동기: {predicted_chiller_params['high_temp_chiller']}")

    print("\n--- 냉동기 최적 가동 대수 산출 (개선된 로직) ---")
    optimal_chiller_units = hvac_optimizer.optimize_chiller_operating_units(predicted_chiller_params)
    print(f"  최적 저온 냉동기 가동 대수: {optimal_chiller_units['optimal_low_temp_chillers_count']} 대")
    print(f"  최적 고온 냉동기 가동 대수: {optimal_chiller_units['optimal_high_temp_chillers_count']} 대")

    print("\n--- 냉각탑 최적 가동 대수 산출 (개선된 로직) ---")
    optimal_ct_units = hvac_optimizer.optimize_cooling_tower_operating_units(optimal_chiller_units, predicted_chiller_params)
    print(f"  최적 냉각탑 가동 대수: {optimal_ct_units['optimal_cooling_towers_count']} 대")
    print("\n=== 예측 및 최적화 완료 ===")

향상된 더미 데이터 생성 및 HVAC 최적화 시뮬레이션 시작 (델타 엔탈피 적용)...


<ipython-input-1-112201492481>:498: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  datetime_index = pd.date_range(start='2023-01-01 00:00:00', periods=num_hours_in_year, freq='H')
<ipython-input-1-112201492481>:525: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_dummy_data[f'{p}FAN_Hz'] = np.random.uniform(35, 55, num_hours_in_year) + np.random.normal(0,2,num_hours_in_year)
<ipython-input-1-112201492481>:529: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_dummy_data[f'{p}예

향상된 더미 데이터 생성 완료: 8760개 행

=== 모델 학습 시작 (델타 엔탈피 적용) ===
OAC 모델 학습 시작 (델타 엔탈피 적용)...
